In [3]:
# Modules
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import datetime

# Functions
from implementations import *
from helpers import *

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed
np.random.seed(1)


(labels_raw, data_raw, ids_raw) = load_csv_data("data/train.csv")
(t_labels, t_data_raw, t_ids) = load_csv_data("data/test.csv")
labels_raw_portion = labels_raw[:10000]
data_raw_portion = data_raw[:10000,:]
ids_raw_portion = ids_raw[:10000]
data_, data_t_, labels = process_data(data_raw_portion, t_data_raw, labels_raw_portion, ids_raw_portion, replace = 'mean')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The original dimensions of the training data set was 10000 samples and 30 columns
 After feature and sample filtering, there are 8976 samples and 23 columns


In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

def cross_val(y, x, k, lambda_, degree):
    """get subsets for cross validation"""
    
    k_indices = build_k_indices(y, k, 0)

    for i in np.arange(k):
        te_indice = k_indices[i]
        tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == i)]
        tr_indice = tr_indice.reshape(-1)
        y_te = y[te_indice]
        y_tr = y[tr_indice]
        x_te = x[te_indice]
        x_tr = x[tr_indice]
        
        # standardize the sets
        
        x_train, mean, variance = standardize(x_tr)
        x_test = standardize_test(x_te, mean, variance)
    
        yield np.array(y_tr), np.array(x_train), np.array(y_te), np.array(x_test) #this is a generator! call next(object) for next set

In [4]:
def least_squares_GD(y, tx, initial_w, tol = 1e-5, max_iters = 10000, gamma = 0.05, k=4,  write = False):
    """Gradient descent algorithm with option for cross validation. 
    
    For cross validation: If k is set to > 0, then the function returns
    the average test loss between the trials, the variance between the trials, the vector of test losses over 
    gradient descent (for each cross validation, so it is a list of lists), the training loss and the 
    last w that was retrieved from the last trial (if one wants to use it for plotting etc)
    
    For retrieving the optimal w: If k is set to 0, the entire y and tx values given to the function will
    be used to calculate the optimal w. the only value that will be returned is w"""
    
    gen = cross_val(y, tx, k, 0.01, 2) # initiate generator object
    test_loss = []
    test_losses_vector = []
    training_loss = []

    # This if is for a final model, that is - if we want to get the best w's (so use all training samples)
    
    if k == 0:
        n_iter = 0
        w = initial_w
        while n_iter < max_iters:
            # compute gradient
            gd = compute_gradient(y, tx, w)
            # compute next w
            w = w - gamma*gd
            n_iter += 1
            
        return w
    
    # This else is for determining if this is a good way to select the model - return test and training errors
    else:
        for i in np.arange(k):
            y_tr, x_tr, y_te, x_te = next(gen)

            # Define parameters to store w and loss
            # ws = [initial_w]
            w = initial_w
            my_losses = [compute_loss(y_tr, x_tr, w)]
            test_mylosses = []
            loss = my_losses[0]
            n_iter=0
            while n_iter < max_iters:
                # compute gradient
                gd = compute_gradient(y_tr, x_tr, w)
                # compute next w
                w = w - gamma*gd
                # compute loss and diff
                loss = compute_loss(y_tr, x_tr, w)
                # store w and loss and increment
                # ws.append(w)
                n_iter += 1
                my_losses.append(loss)
                test_mylosses.append(compute_loss(y_te, x_te, w))
                if abs(my_losses[-1] - my_losses[-2]) < tol:
                    break
                    
                if write == True:
                    print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
            test_losses_vector.append(test_mylosses)
            # append the test_loss to the list so it can be averaged at the end
            test_loss.append(compute_loss(y_te, x_te, w))
            training_loss.append(loss)
            
            
        return np.array(test_loss).mean(), np.array(test_loss).var(), np.array(test_losses_vector), np.array(training_loss).mean(), w


initial_w = np.zeros(data_.shape[1])
test_loss, test_variance, vector_test_loss, training_loss, w = least_squares_GD(labels, data_, initial_w, k = 4, gamma = 0.0005, max_iters = 200) # fit model and retrieve W's across iterations


In [ ]:
means_over_time = vector_test_loss.mean(axis=0)
error1 = abs(means_over_time - vector_test_loss[0])
error2 = abs(means_over_time - vector_test_loss[1])
error3 = abs(means_over_time- vector_test_loss[2])
error4 = abs(means_over_time - vector_test_loss[3])
x = np.arange(len(error1))


In [ ]:
#make a plot of the four k's to see how they vary over time

plt.style.use('seaborn')
# for i in np.arange(4):
#     plt.plot(x, vector_test_loss[i], label= 'Trial {}'.format(i))
plt.plot(x, vector_test_loss[0], label='Trial 1', c='red')
plt.fill_between(x, vector_test_loss[0]-error1, vector_test_loss[0]+error1,
    alpha=0.2, edgecolor='#CC4F1B', facecolor='#FF9848')
plt.plot(x, vector_test_loss[1], label='Trial 2', c = 'green')
plt.fill_between(x, vector_test_loss[1] - error2, vector_test_loss[1] + error2, alpha = 0.2, edgecolor='#FF3F1B', facecolor = '#12E99F')
plt.plot(x, vector_test_loss[2], label='Trial 3', c='blue')
plt.fill_between(x, vector_test_loss[2]-error3, vector_test_loss[2]+error3,
    alpha=0.2, edgecolor='#CC4F1B', facecolor='#12E2FF')
plt.plot(x, vector_test_loss[3], label='Trial 4', c='purple')
plt.fill_between(x, vector_test_loss[3]-error4, vector_test_loss[3]+error4,
    alpha=0.2, edgecolor='#CC4F1B', facecolor='#FF00FF')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Test Set Loss Over Iterations of Gradient Descent')
plt.legend()
plt.show()